In [3]:
import numpy as np

# Define activation functions and their derivatives
def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x)**2

def mse_loss(y_pred, y_true):
    return np.mean((y_pred - y_true)**2)

# Initialize the RNN parameters
input_size = 1      # Input feature size (e.g., a single value in a sequence)
hidden_size = 16    # Number of hidden units
output_size = 1     # Output size (single value prediction)
sequence_length = 10 # Length of each input sequence
learning_rate = 0.01

# Weight matrices
Wxh = np.random.randn(hidden_size, input_size) * 0.01  # Input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 # Hidden to hidden
Why = np.random.randn(output_size, hidden_size) * 0.01 # Hidden to output

# Biases
bh = np.zeros((hidden_size, 1))
by = np.zeros((output_size, 1))

# Forward pass through the RNN for a sequence
def forward(x_seq):
    h_seq = np.zeros((sequence_length, hidden_size, 1))  # Stores hidden states
    y_seq = np.zeros((sequence_length, output_size, 1))  # Stores outputs

    for t in range(sequence_length):
        x = x_seq[t].reshape(-1, 1)  # Get current time step input and reshape for matrix operations
        h_prev = h_seq[t - 1] if t > 0 else np.zeros((hidden_size, 1))  # Use previous hidden state
        h = tanh(np.dot(Wxh, x) + np.dot(Whh, h_prev) + bh)  # Compute hidden state
        y = np.dot(Why, h) + by                               # Compute output

        h_seq[t] = h  # Save hidden state
        y_seq[t] = y  # Save output

    return y_seq, h_seq

# Backpropagation Through Time (BPTT) for the RNN
def backward(x_seq, y_true_seq, y_seq, h_seq):
    global Wxh, Whh, Why, bh, by

    # Gradients initialization
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dh_next = np.zeros((hidden_size, 1))  # Gradient of the next time step's hidden state

    for t in reversed(range(sequence_length)):
        dy = y_seq[t] - y_true_seq[t].reshape(-1, 1)  # Output error
        dWhy += np.dot(dy, h_seq[t].T)
        dby += dy

        dh = np.dot(Why.T, dy) + dh_next  # Backpropagate into hidden state
        dh_raw = dh * tanh_derivative(h_seq[t])  # Apply tanh derivative
        dbh += dh_raw
        dWxh += np.dot(dh_raw, x_seq[t].reshape(1, -1))
        if t > 0:
            dWhh += np.dot(dh_raw, h_seq[t - 1].T)
        dh_next = np.dot(Whh.T, dh_raw)

    # Update weights and biases using gradient descent
    for param, dparam in zip([Wxh, Whh, Why, bh, by], [dWxh, dWhh, dWhy, dbh, dby]):
        param -= learning_rate * dparam

# Training loop
epochs = 100
for epoch in range(epochs):
    # Generate a dummy input sequence and target
    x_seq = np.sin(np.linspace(0, 2 * np.pi, sequence_length))  # Example sequence (sinusoidal)
    y_true_seq = np.roll(x_seq, -1)  # Target is the sequence shifted by one step
    x_seq = x_seq.reshape(sequence_length, 1)  # Reshape for processing
    y_true_seq = y_true_seq.reshape(sequence_length, 1)

    # Forward pass
    y_seq, h_seq = forward(x_seq)

    # Calculate and print loss
    loss = mse_loss(y_seq, y_true_seq)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}")

    # Backward pass
    backward(x_seq, y_true_seq, y_seq, h_seq)


Epoch 1/100, Loss: 0.4500000097644881
Epoch 2/100, Loss: 0.45000002174476367
Epoch 3/100, Loss: 0.45000003869630106
Epoch 4/100, Loss: 0.4500000609061232
Epoch 5/100, Loss: 0.4500000887589745
Epoch 6/100, Loss: 0.45000012274408774
Epoch 7/100, Loss: 0.4500001634638823
Epoch 8/100, Loss: 0.45000021164474235
Epoch 9/100, Loss: 0.450000268150069
Epoch 10/100, Loss: 0.4500003339958349
Epoch 11/100, Loss: 0.45000041036891814
Epoch 12/100, Loss: 0.4500004986485429
Epoch 13/100, Loss: 0.45000060043120677
Epoch 14/100, Loss: 0.45000071755954063
Epoch 15/100, Loss: 0.4500008521556137
Epoch 16/100, Loss: 0.4500010066592748
Epoch 17/100, Loss: 0.45000118387221305
Epoch 18/100, Loss: 0.45000138700851716
Epoch 19/100, Loss: 0.4500016197526266
Epoch 20/100, Loss: 0.45000188632570187
Epoch 21/100, Loss: 0.4500021915615771
Epoch 22/100, Loss: 0.4500025409936306
Epoch 23/100, Loss: 0.45000294095409726
Epoch 24/100, Loss: 0.4500033986875491
Epoch 25/100, Loss: 0.45000392248052307
Epoch 26/100, Loss: 0.4